In [1]:
import cv2
import os
import sys
!pip install xmltodict
import xmltodict
!pip install mtcnn
from mtcnn import MTCNN

trained_face_data = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
"""
YFP:
    train:
        images:
            1
                x.bmp
        xml:
            1
                x.xml
    test:
        images:
            1
                x.bmp
        xml:
            1
                x.xml

TARGET

Grade:
    1:
        x.bmp
    2:
        x.bmp
    3:
        x.bmp
    4:
        x.bmp
"""

path_base = '/content/drive/MyDrive/Final year project/CNN_test/YFP/test/'
xml_base = path_base + 'xml'
target_path = '/content/drive/MyDrive/Final year project/CNN_test/Grade/test/'

# os.makedirs(target_path)

# print(os.listdir(path_base))

detector = MTCNN()

def get_face(image):
    try:
        img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        y, x, h, w = detector.detect_faces(img)[0]['box'] 
        return True, image[x: x + w + 1, y: y + h + 1]
    except:
        return False, -1


def process_class(data):
    observations = []
    # print(data)
    try:
        if type(data['annotation']['object']) == type([]):
            for o in data['annotation']['object']:
                try:
                    observations.append(o['name'])
                except Exception as e:
                    # print(e)
                    continue
        else:
            observations = [data['annotation']['object']['name']]

        if len(observations) == 0:
            return False, '0'
    except Exception as e:
        # print(e)
        return False, '0'

    strong = 0
    for o in observations:
        grade, region = o.split('_')
        if "Strong" in grade:
            strong += 1
      
    if strong == 2:
        result = 3
    else:
        result = 2
    print(strong)
    return True, str(result)


def get_class(folder, image):
    xml_file_path = os.path.join(xml_base, folder, image + '.xml')
    with open(xml_file_path) as f:
        data = f.read()
    data = xmltodict.parse(data)
    return process_class(data)



i=0
for folder in os.listdir(path_base + 'Image/'):
    folder_path = os.path.join(path_base + 'Image/', folder)
    for image in os.listdir(folder_path):
        image_path = os.path.join(folder_path, image)
        val, face = get_face(cv2.imread(image_path))
        print(image_path)
        b, grade = get_class(folder, image[:-4])
        target_class_folder = os.path.join(target_path, grade)
        if b and val:
            cv2.imwrite(os.path.join(target_class_folder, str(i) + '.jpg'), face)
            i += 1